# Part 1: Twitter Network Analysis

_Exercise_ 1: Build the network of retweets.
We will now build a network that has as nodes the Twitter handles of the members of the house, and direct edges between nodes A and B if A has retweeted content posted by B. We will build a weighted network, where the weight of an edge is equal to the number of retweets. You can build the network following the steps below (and you should  be able to reuse many of the functions you wrote in previous weeks):

* Consider the 200 most recent tweets written by each member of the house (use the files [here](https://github.com/suneman/socialgraphs2018/tree/master/files/data_twitter/tweets.zip), or the ones you produced in Part 1). For each file, use a regular expression to find retweets and to extract the Twitter handle of the user whose content was retweeted. All retweets begin with "_RT @originalAuthor:_", where "_originalAuthor_" is the handle of the user whose content was retweeted (and the part of the text you want to extract).
* For each retweet, check if the handle retweeted is the one of a member of the house. If yes, keep it. If no, discard it.
* Use a NetworkX [`DiGraph`](https://networkx.github.io/documentation/development/reference/classes.digraph.html) to store the network. Use weighted edges to account for multiple retweets. Store also the party of each member as a node attribute (use the data in [this file](https://github.com/suneman/socialgraphs2018/blob/master/files/data_twitter/H115_tw.csv), or the data you downloaded in Part 1). Remove self-loops (edges that connect a node with itself).


In [215]:
# imports
import pandas as pd
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2

In [216]:
### Read Data from twitter files into DiGraph###

# File in socialgraphs2018/files/data_twitter/H115_tw.csv
df = pd.read_csv('../../socialgraphs2018/files/data_twitter/H115_tw.csv')

# Creating DiGraph and saving content
contents = {}
DiG = nx.DiGraph()

# Add nodes with attribute 'Party'
for tw_name in df['tw_name']:
    DiG.add_node(tw_name, Party=df[df.tw_name==tw_name]['Party'].values[0])

# Add edges/retweets and save content
pattern = 'RT @\w+'
for tw_name in df['tw_name']:
    
    # Open file and read tweets
    f = open("../../socialgraphs2018/files/data_twitter/tweets/"+tw_name, "r")
    content = f.read()
    
    # Find retweets
    rts_raw = re.findall(pattern, content)
    rts = []
    # Extract tweeter handlers
    for i in range(len(rts_raw)):
        rt = rts_raw[i].replace("RT @", "")
        # Check if tweeter handler a member of the house and is not a self-loop
        if rt in df['tw_name'].values and rt != tw_name:
            rts.append(rt)
            # update weight or add new edge
            if DiG.has_edge(tw_name,rt):
                DiG[tw_name][rt]['weight']+=1
            else:
                DiG.add_edge(tw_name,rt,weight=1)
    
    # save data
    contents[tw_name] = {'name': tw_name,
                         'tweets': content, 
                         'Party': df[df.tw_name==tw_name]['Party'].values[0], 
                         'rt':rts}    

 _Exercise_ 2: Visualize the network of retweets and investigate differences between the parties.
 * Visualize the network using the [Networkx draw function](https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw.html#networkx.drawing.nx_pylab.draw), and nodes coordinates from the force atlas algorithm (see Week 5, Exercise 2). _Hint: use the undirected version of the graph to find the nodes positions for better results, but stick to the directed version for all measurements._ Plot nodes in colors according to their party (e.g. 'red' for republicans and 'blue' for democrats) and set the nodes' size proportional to their total degree. 
  * Compare the network of Retweets with the network of Wikipedia pages (Week 5, exercise 2). Do you observe any difference? How do you explain them?

In [ ]:
G = DiG.to_undirected()

# Edit layout
plt.figure(figsize=(15, 10))
forceatlas2 = ForceAtlas2(
                          # Behavior alternatives
                          outboundAttractionDistribution=False,  # Dissuade hubs
                          linLogMode=False,  # NOT IMPLEMENTED
                          adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                          edgeWeightInfluence=1,

                          # Performance
                          jitterTolerance=0.1,  # Tolerance
                          barnesHutOptimize=True,
                          barnesHutTheta=1.2,
                          multiThreaded=False,  # NOT IMPLEMENTED

                          # Tuning
                          scalingRatio=0.2,
                          strongGravityMode=True,
                          gravity=0.3,

                          # Log
                          verbose=False)

# Divide Democratics and Republicans
Democratics = [key for key, value in contents.items() if value['Party'] == 'Democratic']
Republicans = [key for key, value in contents.items() if value['Party'] == 'Republican'] 

# Calculate size of nodes
node_size_democratics = [G.degree(member)*3 for member in Democratics]
node_size_republicans = [G.degree(member)*3 for member in Republicans]

# Add nodes and edges
positions = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=2000)
nx.draw_networkx_nodes(G, positions, nodelist=Democratics, node_color='blue', node_size=node_size_democratics,cmap=plt.get_cmap('jet'))
nx.draw_networkx_nodes(G, positions, nodelist=Republicans, node_color='red', node_size=node_size_republicans,cmap=plt.get_cmap('jet'))
nx.draw_networkx_edges(G, positions, width=0.2, cmap=plt.get_cmap('jet'))

plt.axis('off')
plt.show()

* Now set the nodes' size proportional to their betweenness centrality. What do you observe?
* Repeat the point above using eigenvector centrality instead. Is there any difference? Can you explain why?
* Who are the three nodes with highest degree within each party? And eigenvector centrality? And betweenness centrality?
* Plot on the same figure the distribution of outgoing strength for the republican and democratic nodes (e.g. the sum of the weight on outgoing links). Which party is more active in retweeting other members of the house?
* Find the 3 members of the republican party that have retweet more often tweets from democratic members. Repeat the measure for the democratic members. Can you explain your results by looking at the Wikipedia pages of these members of the house?

In [246]:
# Add nodes and edges
positions = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=2000)
nx.draw_networkx_nodes(G, positions, nodelist=Democratics, node_color='blue', node_size=nx.betweenness_centrality(DiG),cmap=plt.get_cmap('jet'))
nx.draw_networkx_nodes(G, positions, nodelist=Republicans, node_color='red', node_size=nx.betweenness_centrality(DiG),cmap=plt.get_cmap('jet'))
nx.draw_networkx_edges(G, positions, width=0.2, cmap=plt.get_cmap('jet'))

plt.axis('off')
plt.show()

AttributeError: 'dict' object has no attribute 'sqrt'

_Exercise_ 3: Community detection.

* Use your favorite method of community detection to find communities in the full house of representatives network. Report the value of modularity found by the algorithm. Is it higher or lower than what you found for the Wikipedia network? Comment on your result.
* Visualize the network, using the Force Atlas algorithm (see Lecture 5, exercise 2). This time assign each node a different color based on their _community_. Describe the structure you observe.
* Compare the communities found by your algorithm with the parties by creating a matrix $\mathbf{D}$ with dimension $(B \times C$, where $B$ is the number of parties and $C$ is the number of communities. We set entry $D(i,j)$ to be the number of nodes that party $i$ has in common with community $j$. The matrix $\mathbf{D}$ is what we call a [**confusion matrix**](https://en.wikipedia.org/wiki/Confusion_matrix). 
*  [Plot the confusion matrix](https://scipython.com/book/chapter-7-matplotlib/examples/visualizing-a-matrix-with-imshow/) and explain how well the communities you've detected correspond to the parties. Consider the following questions:
  * Are there any republicans grouped with democrats (and vice versa)?
  * Does the community detection algorithm sub-divide the parties? Do you know anything about American politics that could explain such sub-divisions? Answer in your own words.

# Part 2: What do republican and democratic members tweet about?

_Exercise_ 4: TF-IDF of the republican and democratic tweets.

We will create two documents, one containing the words extracted from tweets of republican members, and the other for Democratic members. We will then use TF-IDF to compare the content of these two documents and create a word-cloud. The procedure you should use is exactly the same you used in exercise 2 of week 7. The main steps are summarized below: 
* Create two large documents, one for the democratic and one for the republican party. Tokenize the pages, and combine the tokens into one long list including all the pages of the members of the same party. 
  * Exclude all twitter handles.
  * Exclude punctuation.
  * Exclude stop words (if you don't know what stop words are, go back and read NLPP1e again).
  * Exclude numbers (since they're difficult to interpret in the word cloud).
  * Set everything to lower case.
  * Compute the TF-IDF for each document.
* Now, create word-cloud for each party. Are these topics less "boring" than the wikipedia topics from a few weeks ago? Why?  Comment on the results.

# Part 3: Sentiment analysis

_Exercise_ 5: Sentiment over the Twitter data.

* Download the LabMT wordlist. It's available as supplementary material from [Temporal Patterns of Happiness and Information in a Global Social Network: Hedonometrics and Twitter](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0026752) (Data Set S1). Describe briefly how the list was generated.
* Based on the LabMT word list, write a function that calculates sentiment given a list of tokens (the tokens should be lower case, etc).
* Create two lists: one including the tweets written by democratic members, and the other including the tweets written by republican members (in the text files, tweets are separated by newlines). 
* Calculate the sentiment of each tweet and plot the distribution of sentiment for each of the two lists. Are there significant differences between the two? Which party post more positive tweets?
* Compute the average _m_ and standard deviation $\sigma$  of the tweets sentiment (considering tweets by both republican and democrats). 
* Now consider only tweets with sentiment lower than m-2$\sigma$. We will refer to them as _negative_ tweets.  Build a list containing the _negative_ tweets written by democrats, and one for republicans. Compute the TF-IDF for these two lists (use the same pre-processing steps in Exercise 5). Create a word-cloud for each of them. Comment on the differences between the _negative_ contents posted by republicans and democrats.
* Repeat the point above, but considering _positive_ tweets instead (e.g. with sentiment larger than m+2$\sigma$). Comment on your results.

In [0]:
import nltk